In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
data_path = "/content/gdrive/My Drive/Data" 
model_path = "/content/gdrive/My Drive/Models"

In [ ]:
train_path = data_path + "/maui-semeval2010-train"
test_path = data_path + "/maui-semeval2010-test"



> Dataset creation (skip it)



In [ ]:
import os
from nltk.tokenize import sent_tokenize

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
txt = sorted([f for f in os.listdir(train_path) if f.endswith(".txt")]) # all .txt files for training
key = sorted([f for f in os.listdir(train_path) if f.endswith(".key")]) # all .key files for training

In [ ]:
test_txt = sorted([f for f in os.listdir(test_path) if f.endswith(".txt")]) # all .txt files for testing
test_key = sorted([f for f in os.listdir(test_path) if f.endswith(".key")]) # all .key files for testing

In [ ]:
key2txt = dict() # dictionary with (key, txt) pairs
for i in range(len(txt)):
  key2txt[key[i]] = txt[i]

In [ ]:
test_key2txt = dict() # dictionary with (key, txt) pairs
for i in range(len(test_txt)):
  test_key2txt[test_key[i]] = test_txt[i]

In [ ]:
def kt2st(key, txt, path): # key, txt -> snts - sentences with keywords, tags - tags for each word
  lines = ""
  for line in open(path + "/" + txt, 'r'):
    lines += " " + line.strip()
  lines = lines.strip()

  tkns = sent_tokenize(lines)
  keys = sorted([line.strip() for line in open(path + "/" + key, 'r')], key=len, reverse=True)
  
  snts = []
  tags = []
  
  for t in tkns:
    tlist = t.lower().split()
    temp = ['O'] * len(tlist)
    for k in keys:
      if k.lower() in t.lower():
        klist = k.lower().split()
        if klist[0] in tlist:
          i0 = tlist.index(klist[0])
          if temp[i0] == 'O':
            temp[i0] = 'B'
            if len(klist) > 1 and len(tlist) >= i0 + len(klist):
              isWrong = False
              for i in range(i0 + 1, i0 + len(klist)):
                if tlist[i] == klist[i - i0]:
                  temp[i] = 'I'
                else:
                  isWrong = True
                  break
              if isWrong:
                for i in range(i0, i0 + len(klist)):
                  temp[i] = 'O'
    
    if set(temp) != {'O'}:
      snts.append(t)
      tags.append(temp)
  return snts, tags

In [ ]:
temp_snts = []
temp_tags = []
for key_, txt_ in key2txt.items():
  snts_, tags_ = kt2st(key_, txt_, train_path)
  temp_snts.append(snts_)
  temp_tags.append(tags_)

In [ ]:
test_temp_snts = []
test_temp_tags = []
for key_, txt_ in test_key2txt.items():
  snts_, tags_ = kt2st(key_, txt_, test_path)
  test_temp_snts.append(snts_)
  test_temp_tags.append(tags_)

In [ ]:
snts = [item for sublist in temp_snts for item in sublist]
tags = [item for sublist in temp_tags for item in sublist]

In [ ]:
test_snts = [item for sublist in test_temp_snts for item in sublist]
test_tags = [item for sublist in test_temp_tags for item in sublist]



> Dataset saving (skip it)



In [ ]:
snts_file = open(data_path + "/snts.txt", 'w')
for s in snts:
  snts_file.write(s + "\n")

In [ ]:
tags_file = open(data_path + "/tags.txt", 'w')
for t in tags:
  tstr = ""
  for subt in t:
    tstr += str(subt)
  tags_file.write(tstr + "\n")

In [ ]:
test_snts_file = open(data_path + "/test_snts.txt", 'w')
for s in test_snts:
  test_snts_file.write(s + "\n")

In [ ]:
test_tags_file = open(data_path + "/test_tags.txt", 'w')
for t in test_tags:
  tstr = ""
  for subt in t:
    tstr += str(subt)
  test_tags_file.write(tstr + "\n")


> Dataset loading (start from here)



In [ ]:
snts = []
tags = []

In [ ]:
snts_file = open(data_path + "/snts.txt", 'r')
for line in snts_file:
  snts.append(line.strip())

In [ ]:
tags_file = open(data_path + "/tags.txt", 'r')
count = 0
for line in tags_file:
  t = []
  tstr = line.strip()
  for subt in tstr:
    t.append(subt)
  count += 1
  tags.append(t)

In [ ]:
test_snts = []
test_tags = []

In [ ]:
test_snts_file = open(data_path + "/test_snts.txt", 'r')
for line in test_snts_file:
  test_snts.append(line.strip())

In [ ]:
test_tags_file = open(data_path + "/test_tags.txt", 'r')
count = 0
for line in test_tags_file:
  t = []
  tstr = line.strip()
  for subt in tstr:
    t.append(subt)
  count += 1
  test_tags.append(t)



> BERT model creation



In [ ]:
pip install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 13.1MB/s 
     |████████████████████████████████| 133kB 24.3MB/s 
     |████████████████████████████████| 6.9MB 11.6MB/s 
     |████████████████████████████████| 71kB 9.4MB/s 
ERROR: botocore 1.19.24 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam



> Hyperparameters



In [ ]:
MAX_LEN = 75
batch_size = 32

In [ ]:
tag2idx = {'B': 2, 'I': 1, 'O': 0}
idx2tag = {2: 'B', 1 : 'I', 0: 'O'}

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



> Preparing dataset



In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

100%|██████████| 231508/231508 [00:00<00:00, 24651931.51B/s]


In [ ]:
token_snts = [tokenizer.tokenize(snt) for snt in snts]

In [ ]:
token_test_snts = [tokenizer.tokenize(snt) for snt in test_snts]

In [ ]:
inp_ids = pad_sequences([tokenizer.convert_tokens_to_ids(token_snt) for token_snt in token_snts], maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
out_tgs = pad_sequences([[tag2idx[t] for t in tag] for tag in tags], maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (770 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (766 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (544 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (630 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (753 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length f

In [ ]:
test_inp_ids = pad_sequences([tokenizer.convert_tokens_to_ids(token_snt) for token_snt in token_test_snts], maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
test_out_tgs = pad_sequences([[tag2idx[t] for t in tag] for tag in test_tags], maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (704 > 512). Running this sequence through BERT will result in indexing errors


In [ ]:
att_masks = [[float(inp_id > 0) for inp_id in inp_idx] for inp_idx in inp_ids]

In [ ]:
#att_masks = [[(0.0 if i == 0 else (0.99999 if i > 0 and t == 0 else 1.0)) for i, t in zip(inp_id, out_tg)] for inp_id, out_tg in zip(inp_ids, out_tgs)]

In [ ]:
train_inp, val_inp, train_out, val_out = train_test_split(inp_ids, out_tgs, random_state=0, test_size=0.1)
train_masks, val_masks, _, _ = train_test_split(att_masks, inp_ids, random_state=0, test_size=0.1)

In [ ]:
train_inp = torch.tensor(train_inp)
val_inp = torch.tensor(val_inp)
train_out = torch.tensor(train_out)
val_out = torch.tensor(val_out)
train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)

In [ ]:
train_data = TensorDataset(train_inp, train_masks, train_out)
train_sampler = RandomSampler(train_data)
train_loader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [ ]:
val_data = TensorDataset(val_inp, val_masks, val_out)
val_sampler = RandomSampler(val_data)
val_loader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [ ]:
test_att_masks = [[float(inp_id > 0) for inp_id in inp_idx] for inp_idx in test_inp_ids]

In [ ]:
test_inp = torch.tensor(test_inp_ids)
test_out = torch.tensor(test_out_tgs)
test_masks = torch.tensor(test_att_masks)

In [ ]:
test_data = TensorDataset(test_inp, test_masks, test_out)
test_sampler = RandomSampler(test_data)
test_loader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

BERT model loading

In [ ]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=(len(tag2idx)))

In [ ]:
model = model.cuda()

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
def accuracy(preds, outs):
  preds_flat = preds.flatten()
  outs_flat = outs.flatten()
  return np.sum(preds_flat == outs_flat) / len(outs_flat)

In [ ]:
def train(epochs, max_norm, params, lr, name, isStorable):
  optimizer = Adam(params, lr=lr)
  best = float('inf')
  for i in trange(epochs, desc='Epoch'):
    # Training
    model.train()
    train_loss = 0
    train_steps = 0

    for batch in train_loader:
      batch = tuple(b.to(device) for b in batch)
      b_inp_ids, b_mask, b_out_tgs = batch

      loss = model(b_inp_ids, token_type_ids=None, attention_mask=b_mask, labels=b_out_tgs)
      loss.backward()

      train_loss += loss.item()
      train_steps += 1

      torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_norm)

      optimizer.step()
      model.zero_grad()
    
    train_loss /= train_steps
    print('Train loss:', train_loss)

    # Validation
    model.eval()
    eval_loss, eval_acc = 0, 0
    eval_steps = 0
    preds, outs = [], []

    for batch in val_loader:
      batch = tuple(b.to(device) for b in batch)
      b_inp_ids, b_mask, b_out_tgs = batch

      with torch.no_grad():
        loss = model(b_inp_ids, token_type_ids=None, attention_mask=b_mask, labels=b_out_tgs)
        logits = model(b_inp_ids, token_type_ids=None, attention_mask=b_mask)
      
      temp_preds = np.argmax(logits.cpu().numpy(), axis=2)
      temp_outs = b_out_tgs.cpu().numpy()
      preds.extend([list(p) for p in temp_preds])
      outs.extend([list(o) for o in temp_outs])

      acc = accuracy(temp_preds, temp_outs)
      
      eval_loss += loss.item()
      eval_acc += acc
      eval_steps += 1

    eval_loss /= eval_steps
    eval_acc /= eval_steps
    print('Val loss:', eval_loss)
    print('Val acc:', eval_acc)
    print('F1-Score:', f1_score([oi for o in outs for oi in o], [pi for p in preds for pi in p], average='weighted'))

    if isStorable and eval_loss < best:
      best = eval_loss
      torch.save(model.state_dict(), model_path + "/key_extractor" + name + ".pt")


> Don't run!!!



In [ ]:
params = [{"params": [p for n, p in list(model.classifier.named_parameters())]}] # partially fine-tuning

In [ ]:
train(epochs=5, max_norm=1.0, params=params, lr=3e-5, name='3e5', isStorable=True)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Train loss: 0.543037089398533
Val loss: 0.29869846473721895
Val acc: 0.9763753770739068
F1-Score: 0.9646194901634687


Epoch:  20%|██        | 1/5 [02:27<09:49, 147.43s/it]

Train loss: 0.2670290276556712
Val loss: 0.2504289255422704
Val acc: 0.9763103318250375
F1-Score: 0.9646319612472058


Epoch:  40%|████      | 2/5 [05:02<07:29, 149.71s/it]

...

Epoch:  60%|██████    | 3/5 [07:40<05:04, 152.23s/it]

Train loss: 0.23376583978781273
Val loss: 0.23493403415469563
Val acc: 0.9761849547511313
F1-Score: 0.9646319612472058


Epoch:  80%|████████  | 4/5 [10:19<02:34, 154.14s/it]

Train loss: 0.228578400473262
Val loss: 0.227884577915949
Val acc: 0.9763282428355957
F1-Score: 0.9646319612472058


Epoch: 100%|██████████| 5/5 [12:58<00:00, 155.71s/it]


In [ ]:
params = [{"params": [p for n, p in list(model.named_parameters())], 'weight_decay_rate': 0.01}] # fully fine-tuning

In [ ]:
train(epochs=5, max_norm=1.0, params=params, lr=3e-5, name='3e5_fully', isStorable=True)







Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Train loss: 0.18764582382781164
Val loss: 0.15021813441725337
Val acc: 0.9559568250377074
F1-Score: 0.9599754992778731








Epoch:  20%|██        | 1/5 [02:42<10:49, 162.34s/it]

Train loss: 0.12525120788436395
Val loss: 0.11471289593507261
Val acc: 0.9608418174962291
F1-Score: 0.9645239663041191








Epoch:  40%|████      | 2/5 [05:26<08:08, 162.88s/it]

...

Train loss: 0.0696668062371471
Val loss: 0.10214261710643768
Val acc: 0.9543514328808447
F1-Score: 0.9626316062827663








Epoch:  80%|████████  | 4/5 [10:55<02:43, 163.61s/it]

Train loss: 0.057400159469722115








Epoch: 100%|██████████| 5/5 [13:37<00:00, 163.51s/it]

Val loss: 0.10701440099407644
Val acc: 0.9507418929110104
F1-Score: 0.9606747427001167




> Run for loading model



In [ ]:
model.load_state_dict(torch.load(model_path + "/key_extractor" + "3e5_fully" + ".pt"))

<All keys matched successfully>

In [ ]:
def accuracy_mean(preds, outs):
  preds_flat = preds.flatten()
  outs_flat = outs.flatten()
  acc = 0
  for tag in list(tag2idx.values()):
    count = 0
    tag_ids = [i for i, o in enumerate(outs_flat) if o == tag]
    for i in tag_ids:
      if preds_flat[i] == tag:
        count += 1
    acc += count / len(tag_ids)
  return acc / len(tag2idx)

In [ ]:
# Testing
def test():
  model.eval()
  eval_loss, eval_acc, eval_acc_mean = 0, 0, 0
  eval_steps = 0
  preds, outs = [], []

  for batch in test_loader:
    batch = tuple(b.to(device) for b in batch)
    b_inp_ids, b_mask, b_out_tgs = batch

    with torch.no_grad():
      loss = model(b_inp_ids, token_type_ids=None, attention_mask=b_mask, labels=b_out_tgs)
      logits = model(b_inp_ids, token_type_ids=None, attention_mask=b_mask)
    
    temp_preds = np.argmax(logits.cpu().numpy(), axis=2)
    temp_outs = b_out_tgs.cpu().numpy()
    preds.extend([list(p) for p in temp_preds])
    outs.extend([list(o) for o in temp_outs])

    acc = accuracy(temp_preds, temp_outs)
    acc_mean = accuracy_mean(temp_preds, temp_outs)
    
    eval_loss += loss.item()
    eval_acc += acc
    eval_acc_mean += acc_mean
    eval_steps += 1

  eval_loss /= eval_steps
  eval_acc /= eval_steps
  eval_acc_mean /= eval_steps
  print('Test loss:', eval_loss)
  print('Test acc:', eval_acc)
  print('Test acc mean:', eval_acc_mean)
  print('F1-Score:', f1_score([oi for o in outs for oi in o], [pi for p in preds for pi in p], average='weighted'))

In [ ]:
test() # for 3e5 model

Test loss: 0.22427435251823158
Test acc: 0.9762249601441711
F1-Score: 0.964472719415991


In [ ]:
test() # for 3e5_fully model

Test loss: 0.19158609764544723
Test acc: 0.9507751091703067
Test acc mean: 0.5708863405454127
F1-Score: 0.95836161857404


In [ ]:
def extract(sent):
  tkns = tokenizer.tokenize(sent)
  ids = torch.tensor([tokenizer.convert_tokens_to_ids(tkns)]).to(device)
  mask = torch.tensor([[0] * len(tkns)]).to(device)
  
  model.eval()
  logit = model(ids, token_type_ids=None, attention_mask=mask).detach().cpu().numpy()
  pred = [list(p) for p in np.argmax(logit, axis=2)]
  for i, tag in enumerate(pred[0]):
    if tag == 2 or tag == 1:
      print(tokenizer.convert_ids_to_tokens(ids[0].cpu().numpy())[i], tag)

In [ ]:
extract("Evaluating Adaptive Resource Management for Distributed Real-Time Embedded Systems")

adaptive 2
resource 1
management 1
